In [109]:
import requests
import pandas as pd

In [115]:
url = 'https://boards.4chan.org/pol/catalog.json'

def get_data(url):

    response = requests.get(url)
    response = response.json()

    data = []
    for items in response:
        page = items['page']
        threads = items['threads']
        for thread in threads:
            keys_to_extract = ['no', 'id', 'sub', 'com', 'replies', 'time', 'last_modified']
            thread_data = {key: thread.get(key) for key in keys_to_extract}
            data.append(thread_data)
    
    return data

df = pd.DataFrame(get_data(url))

In [116]:
df.sort_values(by='replies', ascending=False)

,no,id,sub,com,replies,time,last_modified
157,434296227,IPhY8Bx2,None,My boss discovered I had frauded my covid vacc...,321,1689227323,1689239982
190,434268112,TtKcjD06,aus/pol: AI perp hunter,AI&#039;s picrel of non-race described puffer ...,317,1689205696,1689239220
117,434298876,LUixGhL/,None,"<span class=""quote"">&gt;new nigger trend is ta...",314,1689229707,1689240022
195,434268248,hyALbnCj,Billionaire taking estrogen for longevitiy,Were the trannies right after all?<br>https://...,313,1689205772,1689239290
145,434291559,XCJ5i3TQ,India hate thread,None,312,1689222953,1689239998
...,...,...,...,...,...,...,...
56,434308903,iWleMQFF,Don’t talk about politics,Can we talk about one of the all-time greatest...,0,1689239910,1689239910
55,434308909,U13EWoSu,None,"<span class=""quote"">&gt;Russian MoD posts vide...",0,1689239917,1689239917
39,434308952,lRLg1kAu,why do leftists coop grand movements,such as conserving the environment and end up ...,0,1689239968,1689239968
9,434309014,iusndaCd,None,"Honestly, the British are the absolute scum of...",0,1689240029,1689240029
